In [2]:
import pandas as pd
import numpy as np


def categorize_value(value):
    if value ==0 :
        return "Regular"
    elif  value <=30 :
        return 'SMA0'
    elif  value <=60:
        return 'SMA1'
    elif  value <=90 :
        return "SMA2"
    else:
        return "null"

def IRAC_value(value):
    if value ==0 :
        return 0
    elif  value <=30 :
        return 30
    elif  value <=60:
        return 60
    elif  value <=90 :
        return 60
    else:
        return -1

df_dpd= spark.read.format("delta").load("Tables/dpd")  
df_dpd = df_dpd.toPandas()

df_loan = spark.read.format("delta").load("Tables/loan_master")  
df_loan = df_loan.toPandas()


df_n=df_dpd.drop_duplicates(subset='cust_id')


df_unpivot=pd.melt(df_n,id_vars=['cust_id'],var_name='Month',value_name='Count')

df_unpivot['Date']='28 '+df_unpivot['Month']+' 2023'

df_unpivot['IRACStatus'] = df_unpivot['Count'].astype(int).apply(categorize_value)
 
df_unpivot['IRACValue'] = df_unpivot['Count'].astype(int).apply(IRAC_value)

df_unpivot['custom']=df_unpivot['cust_id'].astype(str)+df_unpivot['Month']
df_merge = df_loan.merge(df_unpivot, on="cust_id", how = 'inner')

# print(df_merge.columns)

sel=df_merge[['cust_id','Month', 'Count', 'Date', 'IRACStatus',
       'IRACValue','name','custom']]



StatementMeta(, 90130261-f7f3-4ea0-8290-37316be8c893, 4, Finished, Available)

In [3]:
sel=spark.createDataFrame(sel)
sel.write.format("delta").mode("overwrite").save("Tables/dpd_data")

StatementMeta(, 90130261-f7f3-4ea0-8290-37316be8c893, 5, Finished, Available)

In [24]:
df_dpd= spark.read.format("delta").load("Tables/dpd_data")
# df_dpd.show()
from pyspark.sql.functions import col,to_timestamp
# df_dpd_new=df_dpd.withColumn("date_column",to_date(col("Date"),"dd MMM yyyy"))
df_dpd_new=df_dpd.withColumn("datetime_column",to_timestamp(col("Date"),"dd MMM yyyy"))
display(df_dpd_new)
df_dpd_new.printSchema()

StatementMeta(, 265becc8-e5d4-4c25-b5d3-d35e5b5c981e, 26, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9e1ed54f-4307-4691-b35f-8f95d19ce33c)

root
 |-- cust_id: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Count: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- IRACStatus: string (nullable = true)
 |-- IRACValue: long (nullable = true)
 |-- name: string (nullable = true)
 |-- custom: string (nullable = true)
 |-- date_column: date (nullable = true)
 |-- datetime_column: timestamp (nullable = true)



In [22]:
df_dpd= spark.read.format("delta").load("Tables/dpd_data")
# df_dpd.show()
from pyspark.sql.functions import col,date_format
df_dpd_new=df_dpd.withColumn("Month_column",date_format(col("date_column"),"MMM"))
df_dpd_new.show()
df_dpd_new.printSchema()

StatementMeta(, 265becc8-e5d4-4c25-b5d3-d35e5b5c981e, 24, Finished, Available)

+-------+-----+-----+-----------+----------+---------+--------------------+----------+-----------+------------+
|cust_id|Month|Count|       Date|IRACStatus|IRACValue|                name|    custom|date_column|Month_column|
+-------+-----+-----+-----------+----------+---------+--------------------+----------+-----------+------------+
|1346418|  Jan|    2|28 Jan 2023|      SMA0|       30|Mullen; Smith and...|1346418Jan| 2023-01-28|         Jan|
|2135813|  Jan|    2|28 Jan 2023|      SMA0|       30|Hull; Skinner and...|2135813Jan| 2023-01-28|         Jan|
|9036935|  Jan|    2|28 Jan 2023|      SMA0|       30|    Wright-Dominguez|9036935Jan| 2023-01-28|         Jan|
|8781683|  Jan|    2|28 Jan 2023|      SMA0|       30|          Suarez Inc|8781683Jan| 2023-01-28|         Jan|
|5345539|  Jan|    2|28 Jan 2023|      SMA0|       30|        Vargas-Ochoa|5345539Jan| 2023-01-28|         Jan|
|5118393|  Jan|    2|28 Jan 2023|      SMA0|       30|      Andrews-Wilson|5118393Jan| 2023-01-28|      

In [25]:
df_dpd_new.write.format("delta").mode("overwrite").option("overwriteSchema", "True").save("Tables/dpd_data")

StatementMeta(, 265becc8-e5d4-4c25-b5d3-d35e5b5c981e, 27, Finished, Available)

In [13]:
from newsapi import NewsApiClient
import csv

# Init
def fetch_news():
    newsapi = NewsApiClient(api_key='f53109aad6644111a5b01d23d6796f23')


    fields = [['Company Name','Alert Type','Alert Category','Alert Date','Keywords','News Link','News HeadLines']]
        
   
    for i in list(keywords_list.keys()):
        # /v2/everything
        all_articles = newsapi.get_top_headlines(q=i,sources='bbc-news,the-verge',verify=False)
        res=[i for i in all_articles['articles']]
        if res==[]:
            all_articles = newsapi.get_everything(q=i,sources='bbc-news',language='en',sort_by='relevancy',verify=False)
            res=[i for i in all_articles['articles']]
        value=res[0]
        fields.append(
            [i.split(' ')[0],keywords_list[i],'EXTERNAL',value['publishedAt'],','.join(i.split('AND')),value['url'],value['title']]
        )

    with open("output.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(fields)
        

keywords_list={
                  'Tesco AND price':'High',

                    'Elon Musk AND sue':'Low',

                    'Prada ':'low',

                    'Amazon AND subscriptions':'Medium',

                    'TikTok OR shuts OR ban':'Medium',

                    'Boohoo':'Medium',

                    'John Lewis AND down AND step AND Dame Sharon':'Low',

                    'Frasers Group AND Mike Ashley AND sell AND Shein':'High',

                    'Microsoft AND Activision Blizzard  ':'High'
               
                }


if __name__ == "__main__": 
    fetch_news()

StatementMeta(, 23c47727-6c60-499c-a6c4-87265035d444, 15, Finished, Available)

ModuleNotFoundError: No module named 'newsapi'

In [14]:
pip freeze


StatementMeta(, 23c47727-6c60-499c-a6c4-87265035d444, 16, Finished, Available)